In [13]:
using Pkg
Pkg.status()

Status `C:\Users\Mishka\Documents\dykes_julia\Project.toml`
  [052768ef] CUDA v4.3.0
  [91a5bcdd] Plots v1.38.12


In [4]:
import Pkg;
Pkg.add("CUDA")

   Resolving package versions...
   Installed JpegTurbo_jll ──────────────── v2.1.91+0
   Installed LERC_jll ───────────────────── v3.0.0+1
   Installed GR_jll ─────────────────────── v0.72.6+0
   Installed libfdk_aac_jll ─────────────── v2.0.2+0
   Installed x265_jll ───────────────────── v3.5.0+0
   Installed Opus_jll ───────────────────── v1.3.2+0
   Installed Libmount_jll ───────────────── v2.35.0+0
   Installed LoggingExtras ──────────────── v1.0.0
   Installed Xorg_xkbcomp_jll ───────────── v1.4.2+4
   Installed Measures ───────────────────── v0.3.2
   Installed RelocatableFolders ─────────── v1.0.0
   Installed Grisu ──────────────────────── v1.0.2
   Installed ConcurrentUtilities ────────── v2.2.0
   Installed Xorg_xcb_util_image_jll ────── v0.4.0+1
   Installed Xorg_xcb_util_wm_jll ───────── v0.4.1+1
   Installed Contour ────────────────────── v0.6.2
   Installed PlotUtils ──────────────────── v1.3.5
   Installed RecipesPipeline ────────────── v0.6.12
   Installed Xorg_libpthr

Precompiling project...
  ✓ LaTeXStrings
  ✓ Pipe
  ✓ TensorCore
  ✓ StatsAPI
  ✓ Contour
  ✓ Measures
  ✓ Compat
  ✓ ConcurrentUtilities
  ✓ Grisu
  ✓ SimpleBufferStream
  ✓ OrderedCollections
  ✓ Formatting
  ✓ NaNMath
  ✓ Unzip
  ✓ URIs
  ✓ DelimitedFiles
  ✓ DataAPI
  ✓ BitFlags
  ✓ LoggingExtras
  ✓ TranscodingStreams
  ✓ RelocatableFolders
  ✓ UnicodeFun
  ✓ OpenSSL_jll
  ✓ Libmount_jll
  ✓ Graphite2_jll
  ✓ Bzip2_jll
  ✓ Xorg_libXau_jll
  ✓ libpng_jll
  ✓ libfdk_aac_jll
  ✓ LAME_jll
  ✓ RecipesBase
  ✓ Pixman_jll
  ✓ LERC_jll
  ✓ fzf_jll
  ✓ JpegTurbo_jll
  ✓ Xorg_libXdmcp_jll
  ✓ Ogg_jll
  ✓ x265_jll
  ✓ x264_jll
  ✓ FixedPointNumbers
  ✓ libaom_jll
  ✓ Zstd_jll
  ✓ Expat_jll
  ✓ LZO_jll
  ✓ Xorg_xtrans_jll
  ✓ Opus_jll
  ✓ Libiconv_jll
  ✓ Libgpg_error_jll
  ✓ Libffi_jll
  ✓ Wayland_protocols_jll
  ✓ Compat → CompatLinearAlgebraExt
  ✓ Xorg_libpthread_stubs_jll
  ✓ Showoff
  ✓ FriBidi_jll
  ✓ Libuuid_jll
  ✓ Missings
  ✓ CodecZlib
  ✓ FreeType2_jll
  ✓ JLFzf
  ✓ libvorbis_jll


In [ ]:
import Pkg;
Pkg.add("CUDA")

In [37]:
collect(devices())

1-element Vector{CuDevice}:
 CuDevice(0): NVIDIA GeForce RTX 3070 Laptop GPU

In [38]:
function kernel()
           dev = Ref{Cint}()
           CUDA.cudaGetDevice(dev)
           @cuprintln("Running on device $(dev[])")
           return
       end

kernel (generic function with 1 method)

In [39]:
kernel()

Running on device 0


In [10]:
using CUDA
CUDA.versioninfo()

CUDA runtime 12.1, artifact installation
CUDA driver 12.0
NVIDIA driver 528.33.0

CUDA libraries: 
- CUBLAS: 12.1.3
- CURAND: 10.3.2
- CUFFT: 11.0.2
- CUSOLVER: 11.4.5
- CUSPARSE: 12.1.0
- CUPTI: 18.0.0
- NVML: 12.0.0+528.33

Julia packages: 
- CUDA.jl: 4.3.0
- CUDA_Driver_jll: 0.5.0+1
- CUDA_Runtime_jll: 0.6.0+0
- CUDA_Runtime_Discovery: 0.2.2

Toolchain:
- Julia: 1.9.0
- LLVM: 14.0.6
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5
- Device capability support: sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80, sm_86

1 device:
  0: NVIDIA GeForce RTX 3070 Laptop GPU (sm_86, 5.780 GiB / 8.000 GiB available)


In [32]:
using Printf

function printDeviceProperties(x)
    @printf("Device:\n");
    @printf("    id:%d\n",x);
    
end

printDeviceProperties (generic function with 1 method)

In [34]:
function print_gpu_properties()

    for (i,device) in enumerate(CUDA.devices())
        println("*** General properties for device $i ***")
        name = CUDA.name(device)
        println("Device name: $name")
        major = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MAJOR)
        minor = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_COMPUTE_CAPABILITY_MINOR)
        println("Compute capabilities: $major.$minor")
        clock_rate = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_CLOCK_RATE)
        println("Clock rate: $clock_rate")
        device_overlap = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_GPU_OVERLAP)
        print("Device copy overlap: ")
        println(device_overlap > 0 ? "enabled" : "disabled")
        kernel_exec_timeout = CUDA.attribute(device, CUDA.CU_DEVICE_ATTRIBUTE_KERNEL_EXEC_TIMEOUT)
        print("Kernel execution timeout: ")
        println(kernel_exec_timeout > 0 ? "enabled" : "disabled")
    end
end

print_gpu_properties (generic function with 1 method)

In [33]:
printDeviceProperties(3)

Device:
    id:3


In [35]:
print_gpu_properties()

*** General properties for device 1 ***
Device name: NVIDIA GeForce RTX 3070 Laptop GPU
Compute capabilities: 8.6
Clock rate: 1560000
Device copy overlap: enabled
Kernel execution timeout: enabled


In [36]:
#Настроить фильтр

#Вывести свойства
print_gpu_properties()

#Настроить



*** General properties for device 1 ***
Device name: NVIDIA GeForce RTX 3070 Laptop GPU
Compute capabilities: 8.6
Clock rate: 1560000
Device copy overlap: enabled
Kernel execution timeout: enabled
